In [1]:
import csv
from ipyleaflet import Map, LayerGroup, CircleMarker

In [2]:
m = Map(zoom=0)
m

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

## Read the GDD grid

In [3]:
with open('grid.csv') as f:
    grid = list(csv.reader(f))
assert grid[0] == ['Latitude', 'Longitude', '1_year average']
grid = grid[1:]

def fix_coord(lat, lon):
    if lon <= 180:
        return lat, lon
    # Some of the points for America are shifted by 360 degrees west
    # This fixes them
    return lat, lon - 360
grid = [fix_coord(float(lat), float(lon)) + (gdd,) for lat, lon, gdd in grid[:]]

grid = {r[:2]: r for r in grid}

## Read the range defenitions

In [4]:
def read_points(file):
    with open(file) as f:
        points = list(csv.reader(f))
    assert points[0] == ['Latitude', 'Longitude']
    return [(float(r[0]), float(r[1])) for r in points[1:]]
    
america = read_points('america.csv')
china = read_points('china.csv')
europe = read_points('europe.csv')

## Draw EAB range on the map

In [5]:
def points_to_layer(points, color):    
    g = LayerGroup()
    for l in points:
        g.add_layer(CircleMarker(location=l, color=color, radius=3))
    return g

layer_america = points_to_layer(america, 'red')
layer_china = points_to_layer(china, 'red')
layer_europe = points_to_layer(europe, 'red')

m += layer_america
m += layer_china
m += layer_europe

## Find nearest grid points

In [6]:
def snap_to_grid(coord, latoff, lonoff):
    lat, lon = coord
    lat = round((lat - latoff) / 0.75) * 0.75 + latoff
    lon = round((lon - lonoff) / 0.75) * 0.75 + lonoff
    return lat, lon

europe_result = [c + grid[snap_to_grid(c, 0, 0)] for c in europe]
china_result = [c + grid[snap_to_grid(c, -0.25, -0.5)] for c in china]

# GDD for America is provided as two grids with different offsets. Try both
def find_in_america(coord):
    result = grid.get(snap_to_grid(coord, 0, 0))
    if result is not None:
        return result
    result = grid.get(snap_to_grid(coord, 0.5, 0.5))
    assert result is not None
    return result

america_result = [c + find_in_america(c) for c in america]

## Draw nearest grid points on the map

In [7]:
layer_america_result = points_to_layer((r[2:4] for r in america_result), 'blue')
layer_china_result = points_to_layer((r[2:4] for r in china_result), 'blue')
layer_europe_result = points_to_layer((r[2:4] for r in europe_result), 'blue')

m += layer_america_result
m += layer_china_result
m += layer_europe_result

## Save the result

In [8]:
def dump(name, data):
    with open(name, 'w') as f:
        w = csv.writer(f)
        w.writerow('lat long gridlat gridlong gdd'.split())
        w.writerows(data)

dump('america_result.csv', america_result)
dump('china_result.csv', china_result)
dump('europe_result.csv', europe_result)